In [ ]:
import pandas as pd
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class Report:
    def __init__(self, report_type, date_range, data):
        self.report_type = report_type
        self.date_range = date_range
        self.data = data
        self.summary = None

    def generate_report(self):
        """Method to be implemented by subclasses to generate specific reports."""
        raise NotImplementedError("Subclasses must implement generate_report method")

    def display_report(self):
        """Displays the report summary in a tabulated format."""
        print(f"\n{'='*80}\n{self.report_type} Report for {self.date_range}\n{'='*80}")
        print(tabulate(self.summary.items(), headers=["Metric", "Value"], tablefmt="grid"))
        self.visualize_report()
    
    def visualize_report(self):
        """Visualizes the report summary using a bar chart."""
        plt.figure(figsize=(10, 5))
        sns.barplot(x=list(self.summary.keys()), y=list(self.summary.values()), palette="viridis")
        plt.xticks(rotation=45, ha='right')
        plt.title(f"{self.report_type} Report for {self.date_range}")
        plt.ylabel("Value")
        plt.xlabel("Metrics")
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

class DailyReport(Report):
    def __init__(self, date, data):
        super().__init__('Daily', date, data)

    def generate_report(self):
        daily_data = self.data[self.data['date'] == self.date_range]
        self.summary = {
            "Total Transactions": daily_data.shape[0],
            "Total Deposits": daily_data[daily_data['transaction_type'] == 'Deposit']['amount'].sum(),
            "Total Withdrawals": daily_data[daily_data['transaction_type'] == 'Withdrawal']['amount'].sum(),
            "Average Transaction Amount": daily_data['amount'].mean()
        }

class WeeklyReport(Report):
    def __init__(self, start_date, end_date, data):
        super().__init__('Weekly', f"{start_date} to {end_date}", data)

    def generate_report(self):
        weekly_data = self.data[(self.data['date'] >= self.date_range.split()[0]) & 
                                (self.data['date'] <= self.date_range.split()[-1])]
        self.summary = {
            "Total Transactions": weekly_data.shape[0],
            "Total Deposits": weekly_data[weekly_data['transaction_type'] == 'Deposit']['amount'].sum(),
            "Total Withdrawals": weekly_data[weekly_data['transaction_type'] == 'Withdrawal']['amount'].sum(),
            "New Accounts Opened": weekly_data['new_accounts'].sum(),
            "Loan Defaults": weekly_data['loan_defaults'].sum()
        }

class MonthlyReport(Report):
    def __init__(self, month, year, data):
        super().__init__('Monthly', f"{month}-{year}", data)
        self.month = int(month)
        self.year = int(year)

    def generate_report(self):
        monthly_data = self.data[(pd.to_datetime(self.data['date']).dt.month == self.month) &
                                 (pd.to_datetime(self.data['date']).dt.year == self.year)]
        self.summary = {
            "Total Transactions": monthly_data.shape[0],
            "Total Deposits": monthly_data[monthly_data['transaction_type'] == 'Deposit']['amount'].sum(),
            "Total Withdrawals": monthly_data[monthly_data['transaction_type'] == 'Withdrawal']['amount'].sum(),
            "New Accounts Opened": monthly_data['new_accounts'].sum(),
            "Total Loans Disbursed": monthly_data['loan_disbursed'].sum(),
            "Total Loan Defaults": monthly_data['loan_defaults'].sum(),
            "Total Revenue": monthly_data['revenue'].sum()
        }

class QuarterlyReport(Report):
    def __init__(self, quarter, year, data):
        super().__init__('Quarterly', f"Q{quarter} {year}", data)
        self.quarter = int(quarter)
        self.year = int(year)

    def generate_report(self):
        quarterly_data = self.data[(pd.to_datetime(self.data['date']).dt.quarter == self.quarter) &
                                   (pd.to_datetime(self.data['date']).dt.year == self.year)]
        self.summary = {
            "Total Transactions": quarterly_data.shape[0],
            "Total Loans Disbursed": quarterly_data['loan_disbursed'].sum(),
            "Total Interest Collected": quarterly_data['interest_collected'].sum(),
            "Net Profit": quarterly_data['revenue'].sum() - quarterly_data['expenses'].sum()
        }
########################################################################################################################################################
# Generate a large dataset with 20 variables
# Replace This With Your Dataset and keep as data
data = pd.DataFrame({
    'date': pd.date_range(start='2025-01-01', periods=500, freq='D').astype(str),
    'transaction_type': np.random.choice(['Deposit', 'Withdrawal'], size=500),
    'amount': np.random.randint(100, 5000, size=500),
    'new_accounts': np.random.randint(0, 5, size=500),
    'loan_disbursed': np.random.randint(0, 10000, size=500),
    'loan_defaults': np.random.randint(0, 10, size=500),
    'interest_collected': np.random.randint(50, 1000, size=500),
    'revenue': np.random.randint(1000, 50000, size=500),
    'expenses': np.random.randint(500, 30000, size=500)
})
########################################################################################################################################################
daily = DailyReport("2025-02-06", data)
daily.generate_report()
daily.display_report()

weekly = WeeklyReport("2025-01-29", "2025-02-04", data)
weekly.generate_report()
weekly.display_report()

monthly = MonthlyReport(1, 2025, data)
monthly.generate_report()
monthly.display_report()

quarterly = QuarterlyReport(1, 2025, data)
quarterly.generate_report()
quarterly.display_report()
